# CompuOpti

In [774]:
import os
import json

import numpy as np
import gurobipy as grb
from gurobipy import GRB

In [775]:
INF = 2 ** 64

In [776]:
model = grb.Model()

In [777]:
instance = "toy_instance.json"

file = os.path.join("instances", instance)
data = json.load(open(file, "r"))

## Modeling

### Parameters of the problem

Define the length of indices

In [778]:
# Number of workers
worker_length = len(data["staff"])

# Number of jobs
job_length = len(data["jobs"])

# Number of skills
skill_length = len(data["qualifications"])

# Number of days
day_length = data["horizon"]

Define jobs parameters

In [779]:
gains_job = np.array([job["gain"] for job in data["jobs"]])
penalties_job = np.array([job["daily_penalty"] for job in data["jobs"]])
due_dates_job = np.array([job["due_date"] for job in data["jobs"]])
work_days_job_skill = np.array([
    [
        job["working_days_per_qualification"][skill] if skill in job["working_days_per_qualification"] else 0
        for skill in data["qualifications"]
    ]
    for job in data["jobs"]
])

Define staff parameters

In [780]:
qualifications_worker_skill = np.array([
    [
        1 if skill in worker["qualifications"] else 0
        for skill in data["qualifications"]
    ]
    for worker in data["staff"]
])
vacations_worker_day = np.array([
    [
        1 if 1 + day in worker["vacations"] else 0
        for day in range(day_length)
    ]
    for worker in data["staff"]
])

### Decision variables

Variable to model the full solution

In [781]:
# 4-D array of binary variables : 1 if a worker is assigned to a certain project for a certain skill on a certain day, else 0
works_worker_job_skill_day = model.addVars(worker_length, job_length, skill_length, day_length, vtype=GRB.BINARY, name="work")

Variables to compute the total gain

In [782]:
is_realized_job = model.addVars(job_length, vtype=GRB.BINARY, name="is_realized") # 1 if a job is realized, else 0

Variable to compute the maximum duration (and the penalties)

In [783]:
started_after_job_day = model.addVars(job_length, day_length, vtype=GRB.BINARY, name="started_after") # 1 if a job is started after a certain day, else 0
finished_before_job_day = model.addVars(job_length, day_length, vtype=GRB.BINARY, name="finished_before") # 1 if a job is finished before a certain day, else 0
max_duration = model.addVar(vtype=GRB.INTEGER, name="max_duration") # Integer that represents the maximum duration for any job

Variables to compute the maximum assignement

In [784]:
is_assigned_worker_job = model.addVars(worker_length, job_length, vtype=GRB.BINARY, name="is_assigned") # 1 if a certain worker is assigned on a certain job, else 0
max_assigned = model.addVar(vtype=GRB.INTEGER, name="max_assigned") # Integer that represents the maximum number of assigned jobs for any worker

### Constraints

#### Time Table Constraints

Define the constraints of the planning problem itself.

- Worker qualification constraint

In [785]:
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= qualifications_worker_skill[worker, skill]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="qualification",
)

{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 2): <gurobi.Constr *Awaiting Model Up

- Uniqueness of the daily assignement : *done with the vacation constraint (see below)*

- Vacation constraint

In [786]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for job in range(job_length) for skill in range(skill_length)) \
            <= 1 - vacations_worker_day[worker, day]
        for worker in range(worker_length)
        for day in range(day_length)
    ),
    name="vacation",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>}

- Job coverage constraint

In [787]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for worker in range(worker_length) for day in range(day_length)) \
            == is_realized_job[job] * work_days_job_skill[job, skill]
        for job in range(job_length)
        for skill in range(skill_length)
    ),
    name="job_coverage",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>}

- Uniqueness of the realized projects : *done with realized_job either 0 or 1*

#### Variable Constraints

Define the constraints that are related to the definition of additional variables.

- Is realized : *done with job coverage constraint*

- Started after / Finished before

*Example of what is expected :* 

| Day             | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|-----------------|---|---|---|---|---|---|---|---|
| Working day     | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 |
| Started after   | 0 | 0 | 1 | 1 | 1 | 1 | 1 | 1 |
| Finished before | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 1 |

In [788]:
# started_after == 0 => works == 0
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= started_after_job_day[job, day]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="started_after",
)

# increasing sequence
model.addConstrs(
    (
        started_after_job_day[job, day] <= started_after_job_day[job, day + 1]
        for job in range(job_length)
        for day in range(day_length - 1)
    ),
    name="started_after_increasing",
)
# is_realized_job == 0 => started_after == 1
model.addConstrs(
    (
        1 - started_after_job_day[job, day] <= is_realized_job[job]
        for job in range(job_length)
        for day in range(day_length)
    ),
    name="started_after_not_realized",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


In [789]:
# finished before == 1 => works == 0
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= 1 - finished_before_job_day[job, day]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="finished_before",
)

# increasing sequence
model.addConstrs(
    (
        finished_before_job_day[job, day] <= finished_before_job_day[job, day + 1]
        for job in range(job_length)
        for day in range(day_length - 1)
    ),
    name="finished_before_increasing",
)
# is_realized_job == 0 => finished_before == 1
model.addConstrs(
    (
        1 - finished_before_job_day[job, day] <= is_realized_job[job]
        for job in range(job_length)
        for day in range(day_length)
    ),
    name="finished_before_not_realized",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


- Maximum duration

In [790]:
model.addConstrs(
    (
        grb.quicksum(
            started_after_job_day[job, day] - finished_before_job_day[job, day]
            for day in range(day_length)
        ) <= max_duration
        for job in range(job_length)
    ),
    name="max_duration",
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

- Is assigned

In [791]:
# exists_skill_day works == 1 => is_assigned == 1
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= is_assigned_worker_job[worker, job]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="is_assigned_worker_job",
)
# forall_skill_day works == 0 => is_assigned == 0
model.addConstrs(
    (
        is_assigned_worker_job[worker, job] <= \
            grb.quicksum(
                works_worker_job_skill_day[worker, job, skill, day]
                for skill in range(skill_length)
                for day in range(day_length)
            )
        for worker in range(worker_length)
        for job in range(job_length)
    ),
    name="is_assigned_worker_job_bis",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>}

- Maximum assigned

In [792]:
model.addConstrs(
    (
        grb.quicksum(is_assigned_worker_job[worker, job] for job in range(job_length)) \
            <= max_assigned
        for worker in range(worker_length)
    ),
    name="max_assigned",
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

### Objectives

In [793]:
# Add primary objective
model.ModelSense = GRB.MAXIMIZE
model.setObjectiveN(
  grb.quicksum(
    gains_job[job] * is_realized_job[job] \
      - penalties_job[job] * grb.quicksum(
        1 - finished_before_job_day[job, day]
        for day in range(due_dates_job[job], day_length)
      )
    for job in range(job_length)
  ),
  0,
  priority=2,
)

# Add multi-objective functions
model.setObjectiveN(
  -max_assigned,
  1,
  priority=1,
)
model.setObjectiveN(
  -max_duration,
  2,
  priority=0,
)

## Optimization

In [794]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1043 rows, 297 columns and 2528 nonzeros
Model fingerprint: 0xc0e99ab1
Variable types: 0 continuous, 297 integer (295 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 818 rows and 152 columns
Presolve time: 0.01s
Presolved: 225 rows and 145 columns
------------------------

In [795]:
# Get optimization status
print(model.Status == GRB.OPTIMAL, model.Status == GRB.TIME_LIMIT, model.Status == GRB.INFEASIBLE)

True False False


In [796]:
# Get objective value
objective_value = model.objVal
print(objective_value)

65.0


In [797]:
import pandas as pd
import matplotlib as mpl

In [798]:
qualifications = data["qualifications"]
names = [worker["name"] for worker in data["staff"]]
jobs = [job["name"] for job in data["jobs"]]

In [799]:
# Give a hex color to each job
cmap = mpl.cm.get_cmap('hsv', int(len(jobs)*1.2))
job_colors = [mpl.colors.rgb2hex(cmap(i)[:3]) for i in range(cmap.N)][:len(jobs)]

In [800]:
def disply_worker_skills():
    res = pd.DataFrame(
        [
            [
                str([q for (r, q) in zip(row_q, qualifications) if r == 1]).replace("'", ""),
                str([d for (r, d) in zip(row_d, range(day_length)) if r == 1]),
            ]
            for row_q, row_d in zip(qualifications_worker_skill, vacations_worker_day)],
        index=names,
        columns=["qualifications", "vacations"]
    )
    return res
    

In [801]:
def highlight_cols(col):
    return [f"color:black;background-color: {color}" for color in job_colors]

def display_work_days():
    res = pd.DataFrame(work_days_job_skill, index=jobs, columns=qualifications)
    res["due date"] = pd.Series(due_dates_job, index=jobs)
    res["gain"] = pd.Series(gains_job, index=jobs)
    res["penalty"] = pd.Series(penalties_job, index=jobs)
    res = res.style.apply(highlight_cols, axis=0)
    return res

In [802]:
def find_task(tab, worker, day):
    for job in range(job_length):
        for skill in range(skill_length):
            if tab[worker, job, skill, day].x == 1:
                return job, qualifications[skill]
    return None

def color_cells(x, df):
        df = df.applymap(lambda val: job_colors[val[0]] if val is not None else "")
        df = df.applymap(lambda color: f"color:{'' if color == '' else 'black'};background-color: {color}")
        return df

def display_time_table():
    data = [
        [
            find_task(works_worker_job_skill_day, worker, day)
            for day in range(day_length)
        ]
        for worker in range(worker_length)
    ]
    df = pd.DataFrame(
        data,
        index=names,
        columns=range(day_length)
    )
    res = df.applymap(lambda x: x[1] if x is not None else None)
    res = res.style.apply(lambda x: color_cells(x, df), axis=None)
    return res

In [803]:
disply_worker_skills()

,qualifications,vacations
Olivia,"[A, B, C]",[]
Liam,"[A, B]",[0]
Emma,[C],[1]


In [804]:
display_work_days()

,A,B,C,due date,gain,penalty
Job1,1,1,1,3,20,3
Job2,1,2,0,3,15,3
Job3,1,0,2,4,15,3
Job4,0,2,1,3,20,3
Job5,0,0,2,5,10,3


In [805]:
display_time_table()

,0,1,2,3,4
Olivia,B,C,B,C,C
Liam,None,A,B,A,None
Emma,C,None,C,C,None


In [806]:
max_duration.x

3.0

In [807]:
max_assigned.x

2.0